In [49]:
import sys
import random

import pandas as pd

from tqdm.notebook import tqdm

In [2]:
# node structure
class node :
    # common prefix
    #  different url
    # address of parent
    # address of left child
    # address of right child
    def __init__(self, i) :
        self.cp = 0
        self.id = i
        self.parent = None
        self.left = None
        self.right = None
        self.height = 1
        self.diffUrl = None

class avl:
    # Calculate height of the avl tree
    def  height(self, N) :
        if (N == None) :
            return  0
        return  N.height
    # calculate balance of the node
    def  getBalance(self, N) :
        if (N == None) :
            return  0
        return  self.height(N.left) - self.height(N.right)
    # function to find common prefix length between two strings
    def  getprefix(self, A,  B) :
        la = len(A)
        lb = len(B)
        c = min(la,lb)
        i = 0
        while (i < c) :
            if (A[i] != B[i]) :
                return  i
            i += 1
        return  c
    #  INSERTION
    def  insert(self, root,  a,  URL) :
        # WHEN ROOT==NULL
        if (root == None) :
            a.diffUrl = URL[a.cp:]
            return  a
        # CALLING RETRIEVE()
        rURL = self.retrieve(root)
        # WHEN NODE IS ALREADY PRESENT
        if (rURL == URL) :
            return  root
        a.cp = self.getprefix(rURL, URL)
        a.parent = root
        if (rURL < URL) : #  GO right
            root.right = self.insert(root.right, a, URL)
        else :#  GO left
            root.left = self.insert(root.left, a, URL)
        root.height = 1 + max(self.height(root.left),self.height(root.right))
        balance = self.getBalance(root)
        rlurl = self.retrieve(root.left)
        rrurl = self.retrieve(root.right)
        #  Left Left Case
        # if (balance > 1 && key < root.left.key)
        if (balance > 1 and (URL < rlurl)) :
            return  self.rightRotate(root)
        #  Right Right Case
        # if (balance < -1 && key > root.right.key)
        if (balance < -1 and (URL > rrurl)) :
            return  self.leftRotate(root)
        #  Left Right Case
        #  if (balance > 1 && key > root.left.key)
        if (balance > 1 and (URL > rlurl)) :
            root.left = self.leftRotate(root.left)
            return  self.rightRotate(root)
        #  Right Left Case
        # if (balance < -1 && key < root.right.key)
        if (balance < -1 and (URL < rrurl)) :
            root.right = self.rightRotate(root.right)
            return  self.leftRotate(root)
        return  root
    # LEFT_ROTATION
    def  leftRotate(self, x) :
        xurl = self.retrieve(x)
        y = x.right
        yurl = self.retrieve(y)
        y.parent = x.parent
        x.parent = y
        x.right = y.left
        y.left = x
        # CALCULATE HEIGHT
        x.height = max(self.height(x.left),self.height(x.right)) + 1
        y.height = max(self.height(y.left),self.height(y.right)) + 1
        # CALL RECALC() FOR Y AND X
        y = self.recalc(y, yurl)
        x = self.recalc(x, xurl)
        return  y
    # RIGHT_ROTATION
    def  rightRotate(self, x) :
        xurl = self.retrieve(x)
        y = x.left
        yurl = self.retrieve(y)
        y.parent = x.parent
        x.parent = y
        x.left = y.right
        y.right = x
        # CALCULATE HEIGHT
        x.height = max(self.height(x.left),self.height(x.right)) + 1
        y.height = max(self.height(y.left),self.height(y.right)) + 1
        # CALL RECALC() FOR Y AND X
        y = self.recalc(y, yurl)
        x = self.recalc(x, xurl)
        return  y
    # RECALCULATE COMMON PREFIX AND DIFFERENT URL
    def  recalc(self, a,  url) :
        purl = self.retrieve(a.parent)
        a.cp = self.getprefix(purl, url)
        a.diffUrl = url[a.cp:]
        return  a
    # SEARCHING
    def  search(self, root,  URL) :
        if (root == None) :
            return  None
        temp = self.retrieve(root)
        if (temp == URL) :
            return  root
        if (temp < URL) : # go right
            return  self.search(root.right, URL)
        # go left
        return  self.search(root.left, URL)
    # RETREIVING
    def  retrieve(self, a) :
        if (a == None) :
            return  ""
        if (a.cp == 0) :
            return  a.diffUrl
        t = (self.retrieve(a.parent))[0:a.cp] + a.diffUrl
        return  t

    
    def preOrder(self, root):
 
        if not root:
            return
 
        print("{0} ".format(root.diffUrl), end="")
        self.preOrder(root.left)
        self.preOrder(root.right)

In [3]:
#dataset containing urls from bbc.com website
df = pd.read_csv('../../osama/Downloads/bbc_sitemaps.csv')

In [4]:
#check number of urls (~4 million urls)
df['loc']

0          https://www.bbc.com/arabic/middleeast/2009/06/...
1          https://www.bbc.com/arabic/middleeast/2009/06/...
2          https://www.bbc.com/arabic/business/2009/06/09...
3          https://www.bbc.com/arabic/multimedia/2009/06/...
4          https://www.bbc.com/arabic/business/2009/06/09...
                                 ...                        
3995807    https://www.bbc.com/persian/tv-and-radio-52114329
3995808           https://www.bbc.com/burmese/burma-52116369
3995809           https://www.bbc.com/persian/sport-52099005
3995810    https://www.bbc.com/zhongwen/trad/multimedia/2...
3995811        https://www.bbc.com/vietnamese/world-52102027
Name: loc, Length: 3995812, dtype: object

In [24]:
urls = list(df['loc'])

In [ ]:
avl_tree = avl()
root = None
indexes = []
id = 0

for u in tqdm(urls):
    a = node(id)
    indexes.append(a)
    id = id + 1
    root = avl_tree.insert(root, a, u)

In [27]:
total_size_uncompressed = 0
for s in urls:
    total_size_uncompressed = total_size_uncompressed + sys.getsizeof(s)

In [39]:
uncompressed_size_in_mb = total_size_uncompressed / 1024**2
print(f'Total size before compression: {round(uncompressed_size_in_mb, 2)} MB')

Total size before compression: 405.66 MB


In [30]:
total_size_avl_compressed = 0
for s in indexes:
    total_size_avl_compressed = total_size_avl_compressed + sys.getsizeof(s)

In [38]:
total_size_compressed = total_size_avl_compressed / 1024**2
print(f'Total size after compression: {round(total_size_compressed, 2)} MB')

Total size after compression: 182.91 MB


In [41]:
sapce_saving = (1 - total_size_avl_compressed/total_size_uncompressed) * 100
print(str(round(sapce_saving, 2)) + " % sapce saving")

54.91 % sapce saving


In [64]:
#See how a random url is stored
random_urls = random.sample(urls, 3)
for url in random_urls:
    node = avl_tree.search(root, url)
    print(f'\'{url}\' STORED_AS \'{node.diffUrl}\'')

'https://www.bbc.com/news/world-africa-12421747' STORED_AS '1747'
'https://www.bbc.com/turkce/haberler/2012/05/120508_end_austerity' STORED_AS 'end_austerity'
'https://www.bbc.com/ukrainian/news_in_brief/2014/11/141111_vs_poroshenko_biden' STORED_AS '1_vs_poroshenko_biden'
